# Scrape Amazon, Helium 10, and Etsy

## Import libraries

In [1]:
# import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
import urllib.request
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import locale
from config import email, password
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

## Create class to pull data

In [79]:
# Create class for scraping
class amazon_h10():
    def __init__(self, search):
        print(f'You searched for {search}')
    def complete_scrape(self, search):
        print('you are scraping amazon')
        amazon_url = "https://www.amazon.com"
        # open the driver
        options = Options()
        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(30)
        # scrape amazon      
        search_df = pd.DataFrame()
        search_df1 = pd.DataFrame()
        amazon_df = pd.DataFrame()  
        for word in search:
            search_df1['search_term'] = [word]
            search_df = search_df.append(search_df1)
            try:
                search_url = f'https://www.amazon.com/s?k={word}&ref=nb_sb_noss_1'
                driver.get(search_url)
                html = driver.page_source
                search_beautify = BeautifulSoup(html, 'html.parser')
                data = search_beautify.findAll('div', class_="a-section a-spacing-none")
                try:
                    link = [link.a['href'] for link in data]
                except:
                    pass

                data2 = search_beautify.findAll('span', class_="a-offscreen")
                time.sleep(3)
                try:
                    amz_prod_price = [x.text.split('$')[1] for x in data2]
                except:
                    pass
                amz_prod_price = amz_prod_price[:10]
                amz_prod_name = []

                for i in range(12,22):
                    product_link = amazon_url + link[i]
                    break_link = product_link.split('/')
                    title = break_link[3].replace('-', ' ')
                    amz_prod_name.append(title)
                amazon_df1 = pd.DataFrame()  
                amazon_df1['product_name'] = amz_prod_name
                amazon_df1['product_price'] = amz_prod_price
                amazon_df1['search_term'] = [f'{word}' for count in amz_prod_name]
                amazon_df = amazon_df.append(amazon_df1)
            except:
                pass
#     ==================================
        print('you are scraping helium')
        # sign into Helium 10
        driver.get('https://members.helium10.com/user/signin')
        login = driver.find_element_by_id('loginform-email')
        login.send_keys(email)
        password_route = driver.find_element_by_id('loginform-password')
        password_route.send_keys(password)
        login_button = driver.find_elements_by_xpath("//button[contains(text(), 'Log In')]")
        login_button[0].click()
        time.sleep(3)
        
        # create global data frame
        overview_data_df = pd.DataFrame()
        top5_monthly_revenue_df = pd.DataFrame()
        overview_data_df1 = pd.DataFrame()
        top5_monthly_revenue_df1 = pd.DataFrame()
        
        # type in the key words search
        for words in search:
            #  link for niche
            h_10_niche = "https://members.helium10.com/black-box/niche"
            # visit link
            driver.get(h_10_niche)
            search_bar = driver.find_element_by_id('filter-asin')
            search_bar.clear()
            search_bar.send_keys(words)
            
            try:
                # click on search button
                time.sleep(1)
                search_button = driver.find_elements_by_xpath("//button[contains(text(), 'Search')]")
                search_button[0].click()
            except:
                time.sleep(2)
                search_button = driver.find_elements_by_xpath("//button[contains(text(), 'Search')]")
                search_button[0].click()

            # Scrape overview data    
            try: 
                overview_data = driver.find_elements_by_xpath("//div[@class='col-xs-12 col-sm-6 col-md-2']")

                col1 = []
                col2 = []
                time.sleep(1)
                for i in range(0,5):
                    h_10_data = overview_data[i].text
                    h_data = h_10_data.split('\n')
                    col1.append(h_data[0].strip())
                    value = h_data[1].split('$')
                    if len(value) > 1:
                        value = value[1]
                        col2.append(value)
                    else:
                        col2.append(value[0])
                # pull data from 20-24
                overview_data_df1['revenue_specs'] = col1
                overview_data_df1['revenue_value'] = col2
                overview_data_df1['search_term'] = [f'{words}' for x in col1]
                overview_data_df = overview_data_df.append(overview_data_df1)

                # Pull revenue data  
                sort_product_list = Select(driver.find_element_by_id('sort'))
                driver.find_element_by_xpath("//select[@name='sort']/option[text()='Monthly Revenue High To Low']").click()

                # pull in data
                # product title
                time.sleep(1)
                product_name = driver.find_elements_by_class_name('media-heading')
                product_names = [name.text for name in product_name]

                # price
                product_price = driver.find_elements_by_class_name('price-chart')
                product_prices = [price.text.split('$')[1] for price in product_price]

                # monthly sales
                product_sales_monthly = driver.find_elements_by_class_name('monthlySales-column')
                product_sales_monthly_s = [sales.text for sales in product_sales_monthly]

                # monthly revenue 
                product_revenue_monthly = driver.find_elements_by_class_name('monthlyRevenue-column')
                product_revenue_monthly_s = [rev.text.split('$')[1] for rev in product_revenue_monthly]

                # put into a df
                top5_monthly_revenue_df1['monthly_product'] = product_names[:5]
                top5_monthly_revenue_df1['monthly_price'] = product_prices[:5]
                top5_monthly_revenue_df1['monthly_sales'] = product_sales_monthly_s[:5]
                top5_monthly_revenue_df1['monthly_revenue'] = product_revenue_monthly_s[:5]
                top5_monthly_revenue_df1['search_term'] = [f'{words}' for x in product_names[:5]]
                top5_monthly_revenue_df = top5_monthly_revenue_df.append(top5_monthly_revenue_df1)
            except:
                pass

#     ====================
        etsy_df = pd.DataFrame()
        etsy_df1 = pd.DataFrame()
        print("you are scraping etsy")
        # loop over search words        
        for word in search:
            # visit url
            search_url = f'https://www.etsy.com/search?q={word}&explicit=1&order=most_relevant'
            driver.get(search_url)
            
            # product name
            try:
                product = driver.find_elements_by_class_name("v2-listing-card__info")
                product_name = [link.text.strip() for link in product]

                # produce price
                price = driver.find_elements_by_class_name("n-listing-card__price")
                product_price = [x.text.strip() for x in price]
                # clean data 
                n = [x.split('\n') for x in product_price]
                product_price = [y[0] for y in n]
                b = [x.split('$') for x in product_price]
                locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 
                try:
                    product_price = [locale.atof(y[1]) for y in b]
                except:
                    product_price = [y[1] for y in b]

                # df to hold the data    
                etsy_df1['product_name'] = product_name[:5]
                etsy_df1['product_price'] = product_price[:5]
                etsy_df1['search_term'] = [f'{word}' for x in product_name[:5]]
                etsy_df = etsy_df.append(etsy_df1)
            except:
                pass
        driver.close()
        print("scraping complete")
        search_df = search_df.set_index(['search_term'])
        amazon_df = amazon_df.set_index(['search_term'])
        overview_data_df = overview_data_df.set_index(['search_term'])
        top5_monthly_revenue_df = top5_monthly_revenue_df.set_index(['search_term'])
        etsy_df = etsy_df.set_index(['search_term'])
        return [search_df, amazon_df, overview_data_df, top5_monthly_revenue_df, etsy_df]

    def amazon(self, search):
        print('you are scraping amazon')
        amazon_url = "https://www.amazon.com"
        # open the driver
        options = Options()
        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(30)
        # scrape amazon      
        search_df = pd.DataFrame()
        search_df1 = pd.DataFrame()
        amazon_df = pd.DataFrame()  
        for word in search:
            search_df1['search_term'] = [word]
            search_df = search_df.append(search_df1)
            try:
                search_url = f'https://www.amazon.com/s?k={word}&ref=nb_sb_noss_1'
                driver.get(search_url)
                html = driver.page_source
                search_beautify = BeautifulSoup(html, 'html.parser')
                data = search_beautify.findAll('div', class_="a-section a-spacing-none")
                try:
                    link = [link.a['href'] for link in data]
                except:
                    pass

                data2 = search_beautify.findAll('span', class_="a-offscreen")
                time.sleep(3)
                try:
                    amz_prod_price = [x.text.split('$')[1] for x in data2]
                except:
                    pass
                amz_prod_price = amz_prod_price[:10]
                amz_prod_name = []

                for i in range(12,22):
                    product_link = amazon_url + link[i]
                    break_link = product_link.split('/')
                    title = break_link[3].replace('-', ' ')
                    amz_prod_name.append(title)
                amazon_df1 = pd.DataFrame()  
                amazon_df1['product_name'] = amz_prod_name
                amazon_df1['product_price'] = amz_prod_price
                amazon_df1['search_term'] = [f'{word}' for count in amz_prod_name]
                amazon_df = amazon_df.append(amazon_df1)
            except:
                pass
        driver.close()
        amazon_df = amazon_df.set_index(['search_term'])
        return amazon_df
        
    def helium(self, search):
        print('you are scraping helium')
        # sign into Helium 10
        driver.get('https://members.helium10.com/user/signin')
        login = driver.find_element_by_id('loginform-email')
        login.send_keys(email)
        password_route = driver.find_element_by_id('loginform-password')
        password_route.send_keys(password)
        login_button = driver.find_elements_by_xpath("//button[contains(text(), 'Log In')]")
        login_button[0].click()
        time.sleep(3)
        
        # create global data frame
        overview_data_df = pd.DataFrame()
        top5_monthly_revenue_df = pd.DataFrame()
        overview_data_df1 = pd.DataFrame()
        top5_monthly_revenue_df1 = pd.DataFrame()
        
        # type in the key words search
        for words in search:
            #  link for niche
            h_10_niche = "https://members.helium10.com/black-box/niche"
            # visit link
            driver.get(h_10_niche)
            search_bar = driver.find_element_by_id('filter-asin')
            search_bar.clear()
            search_bar.send_keys(words)
            
            try:
                # click on search button
                time.sleep(1)
                search_button = driver.find_elements_by_xpath("//button[contains(text(), 'Search')]")
                search_button[0].click()
            except:
                time.sleep(2)
                search_button = driver.find_elements_by_xpath("//button[contains(text(), 'Search')]")
                search_button[0].click()

            # Scrape overview data    
            try: 
                overview_data = driver.find_elements_by_xpath("//div[@class='col-xs-12 col-sm-6 col-md-2']")

                col1 = []
                col2 = []
                time.sleep(1)
                for i in range(0,5):
                    h_10_data = overview_data[i].text
                    h_data = h_10_data.split('\n')
                    col1.append(h_data[0].strip())
                    value = h_data[1].split('$')
                    if len(value) > 1:
                        value = value[1]
                        col2.append(value)
                    else:
                        col2.append(value[0])
                # pull data from 20-24
                overview_data_df1['revenue_specs'] = col1
                overview_data_df1['revenue_value'] = col2
                overview_data_df1['search_term'] = [f'{words}' for x in col1]
                overview_data_df = overview_data_df.append(overview_data_df1)

                # Pull revenue data  
                sort_product_list = Select(driver.find_element_by_id('sort'))
                driver.find_element_by_xpath("//select[@name='sort']/option[text()='Monthly Revenue High To Low']").click()

                # pull in data
                # product title
                time.sleep(1)
                product_name = driver.find_elements_by_class_name('media-heading')
                product_names = [name.text for name in product_name]

                # price
                product_price = driver.find_elements_by_class_name('price-chart')
                product_prices = [price.text.split('$')[1] for price in product_price]

                # monthly sales
                product_sales_monthly = driver.find_elements_by_class_name('monthlySales-column')
                product_sales_monthly_s = [sales.text for sales in product_sales_monthly]

                # monthly revenue 
                product_revenue_monthly = driver.find_elements_by_class_name('monthlyRevenue-column')
                product_revenue_monthly_s = [rev.text.split('$')[1] for rev in product_revenue_monthly]

                # put into a df
                top5_monthly_revenue_df1['monthly_product'] = product_names[:5]
                top5_monthly_revenue_df1['monthly_price'] = product_prices[:5]
                top5_monthly_revenue_df1['monthly_sales'] = product_sales_monthly_s[:5]
                top5_monthly_revenue_df1['monthly_revenue'] = product_revenue_monthly_s[:5]
                top5_monthly_revenue_df1['search_term'] = [f'{words}' for x in product_names[:5]]
                top5_monthly_revenue_df = top5_monthly_revenue_df.append(top5_monthly_revenue_df1)
            except:
                pass
        driver.close()
        overview_data_df = overview_data_df.set_index(['search_term'])
        top5_monthly_revenue_df = top5_monthly_revenue_df.set_index(['search_term'])
        return [overview_data_df, top5_monthly_revenue_df]
    def etsy(self, search):
        options = Options()
        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(30)
        etsy_df = pd.DataFrame()
        etsy_df1 = pd.DataFrame()
        print("you are scraping etsy")
        # loop over search words        
        for word in search:
            # visit url
            search_url = f'https://www.etsy.com/search?q={word}&explicit=1&order=most_relevant'
            driver.get(search_url)
            
            # product name
            try:
                product = driver.find_elements_by_class_name("v2-listing-card__info")
                product_name = [link.text.strip() for link in product]

                # produce price
                price = driver.find_elements_by_class_name("n-listing-card__price")
                product_price = [x.text.strip() for x in price]
                # clean data 
                n = [x.split('\n') for x in product_price]
                product_price = [y[0] for y in n]
                b = [x.split('$') for x in product_price]
                locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 
                try:
                    product_price = [locale.atof(y[1]) for y in b]
                except:
                    product_price = [y[1] for y in b]

                # df to hold the data    
                etsy_df1['product_name'] = product_name[:5]
                etsy_df1['product_price'] = product_price[:5]
                etsy_df1['search_term'] = [f'{word}' for x in product_name[:5]]
                etsy_df = etsy_df.append(etsy_df1)
            except:
                pass
        driver.close()
        print("scraping complete")
        etsy_df = etsy_df.set_index(['search_term'])
        driver.close()
        return [etsy_df]



In [ ]:
#  original
# import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import pymongo
from splinter import Browser
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
import urllib.request
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import locale

class amazon_h10():
    def __init__(self, search):
        print(f'You searched for {search}')
    def complete_scrape(self, search):
        print('you are scraping amazon')
        amazon_url = "https://www.amazon.com"
        # open the driver
        options = Options()
        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(30)
        # scrape amazon      
        amazon_df = pd.DataFrame()  
        for word in search:
            
            try:
                search_url = f'https://www.amazon.com/s?k={word}&ref=nb_sb_noss_1'
                driver.get(search_url)
                html = driver.page_source
                search_beautify = BeautifulSoup(html, 'html.parser')
                data = search_beautify.findAll('div', class_="a-section a-spacing-none")
                try:
                    link = [link.a['href'] for link in data]
                except:
                    pass

                data2 = search_beautify.findAll('span', class_="a-offscreen")
                time.sleep(3)
                try:
                    amz_prod_price = [x.text.split('$')[1] for x in data2]
                except:
                    pass
                amz_prod_price = amz_prod_price[:10]
                amz_prod_name = []

                for i in range(12,22):
                    product_link = amazon_url + link[i]
                    break_link = product_link.split('/')
                    title = break_link[3].replace('-', ' ')
                    amz_prod_name.append(title)
                amazon_df1 = pd.DataFrame()  
                amazon_df1['product_name'] = amz_prod_name
                amazon_df1['product_price'] = amz_prod_price
                amazon_df1['search_term'] = [f'{word}' for count in amz_prod_name]
                amazon_df = amazon_df.append(amazon_df1)
            except:
                pass
#     ==================================
        print('you are scraping helium')
        # sign into Helium 10
        driver.get('https://members.helium10.com/user/signin')
        login = driver.find_element_by_id('loginform-email')
        login.send_keys(email)
        password_route = driver.find_element_by_id('loginform-password')
        password_route.send_keys(password)
        login_button = driver.find_elements_by_xpath("//button[contains(text(), 'Log In')]")
        login_button[0].click()
        time.sleep(3)
        
        # create global data frame
        overview_data_df = pd.DataFrame()
        top5_monthly_revenue_df = pd.DataFrame()

        # type in the key words search
        for words in search:
            #  link for niche
            h_10_niche = "https://members.helium10.com/black-box/niche"
            # visit link
            driver.get(h_10_niche)
            search_bar = driver.find_element_by_id('filter-asin')
            search_bar.clear()
            search_bar.send_keys(words)
            
            # click on search button
            search_button = driver.find_elements_by_xpath("//button[contains(text(), 'Search')]")
            search_button[0].click()

            # Scrape overview data    
            try: 
                overview_data = driver.find_elements_by_xpath("//div[@class='col-xs-12 col-sm-6 col-md-2']")

                col1 = []
                col2 = []
                time.sleep(1)
                for i in range(0,5):
                    h_10_data = overview_data[i].text
                    h_data = h_10_data.split('\n')
                    col1.append(h_data[0].strip())
                    value = h_data[1].split('$')
                    if len(value) > 1:
                        value = value[1]
                        col2.append(value)
                    else:
                        col2.append(value[0])
                # pull data from 20-24
                overview_data_df['revenue_specs'] = col1
                overview_data_df['revenue_value'] = col2
                overview_data_df['search_term'] = [f'{word}' for count in col1]

                # Pull revenue data  
                sort_product_list = Select(driver.find_element_by_id('sort'))
                driver.find_element_by_xpath("//select[@name='sort']/option[text()='Monthly Revenue High To Low']").click()

                # pull in data
                # product title
                time.sleep(1)
                product_name = driver.find_elements_by_class_name('media-heading')
                product_names = [name.text for name in product_name]

                # price
                product_price = driver.find_elements_by_class_name('price-chart')
                product_prices = [price.text.split('$')[1] for price in product_price]

                # monthly sales
                product_sales_monthly = driver.find_elements_by_class_name('monthlySales-column')
                product_sales_monthly_s = [sales.text for sales in product_sales_monthly]

                # monthly revenue 
                product_revenue_monthly = driver.find_elements_by_class_name('monthlyRevenue-column')
                product_revenue_monthly_s = [rev.text.split('$')[1] for rev in product_revenue_monthly]

                # put into a df
                top5_monthly_revenue_df['monthly_product'] = product_names[:5]
                top5_monthly_revenue_df['monthly_price'] = product_prices[:5]
                top5_monthly_revenue_df['monthly_sales'] = product_sales_monthly_s[:5]
                top5_monthly_revenue_df['monthly_revenue'] = product_revenue_monthly_s[:5]
                top5_monthly_revenue_df['search_term'] = [f'{word}' for count in product_names[:5]]
            except:
                pass

#     ====================
        etsy_df = pd.DataFrame()
        print("you are scraping etsy")
        # loop over search words        
        for word in search:
            # visit url
            search_url = f'https://www.etsy.com/search?q={word}&explicit=1&order=most_relevant'
            driver.get(search_url)
            
            # product name
            try:
                product = driver.find_elements_by_class_name("v2-listing-card__info")
                product_name = [link.text.strip() for link in product]

                # produce price
                price = driver.find_elements_by_class_name("n-listing-card__price")
                product_price = [x.text.strip() for x in price]
                # clean data 
                n = [x.split('\n') for x in product_price]
                product_price = [y[0] for y in n]
                b = [x.split('$') for x in product_price]
                locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 
                try:
                    product_price = [locale.atof(y[1]) for y in b]
                except:
                    product_price = [y[1] for y in b]

                # df to hold the data    
                etsy_df['product_name'] = product_name[:5]
                etsy_df['product_price'] = product_price[:5]
                etsy_df['search_term'] = [f'{word}' for count in product_name[:5]]
            except:
                pass
        driver.close()
        print("scraping complete")
        return [amazon_df, overview_data_df, top5_monthly_revenue_df, etsy_df]
    def amazon(self, search):
        print('you are scraping amazon')
                # open the driver
        # unpacked_extension_path = '/Users/prettyvo/Library/Application Support/Google/Chrome/Default/Extensions/njmehopjdpcckochcggncklnlmikcbnb/4.2_0'
        options = Options()
        # options.add_argument('--load-extension={}'.format(unpacked_extension_path))
        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(30)
        # scrape amazon 
                # open up amazon        
        amazon_url = "https://www.amazon.com"
        driver.get(amazon_url)
        amazon_df = pd.DataFrame()   
        for word in search:
            search_url = f'https://www.amazon.com/s?k={word}&ref=nb_sb_noss_1'
            driver.get(search_url)
            # time.sleep(3)
            html = driver.page_source
            search_beautify = BeautifulSoup(html, 'html.parser')
            data = search_beautify.findAll('div', class_="a-section a-spacing-none")
            try:
                link = [link.a['href'] for link in data]
            except:
                pass
            
            data2 = search_beautify.findAll('span', class_="a-offscreen")
            time.sleep(3)
            try:
                amz_prod_price = [x.text.split('$')[1] for x in data2]
            except:
                pass
            amz_prod_price = amz_prod_price[:10]
            amz_prod_name = []
            amz_prod_asin = []

            for i in range(12,22):
                product_link = amazon_url + link[i]
                break_link = product_link.split('/')
                title = break_link[3].replace('-', ' ')
                amz_prod_name.append(title)
                amz_prod_asin.append(break_link[5])

            amazon_df[f'{word}_name'] = amz_prod_name   
            amazon_df[f'{word}_price'] = amz_prod_price
            amazon_df[f'{word}_asin'] = amz_prod_asin
        driver.close()
        return amazon_df
        
    def helium(self, search):
        print('you are scraping helium')
        # open the driver
        # unpacked_extension_path = '/Users/prettyvo/Library/Application Support/Google/Chrome/Default/Extensions/njmehopjdpcckochcggncklnlmikcbnb/4.2_0'
        options = Options()
        # options.add_argument('--load-extension={}'.format(unpacked_extension_path))
        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(30)

        # sign into Helium 10
        driver.get('https://members.helium10.com/user/signin')
        login = driver.find_element_by_id('loginform-email')
        login.send_keys('justin@breathingcolor.com')
        password = driver.find_element_by_id('loginform-password')
        password.send_keys('TjDxAfM0w2m')
        login_button = driver.find_elements_by_xpath("//button[contains(text(), 'Log In')]")
        login_button[0].click()
        time.sleep(5)
        
        # #  link for niche
        # h_10_niche = "https://members.helium10.com/black-box/niche"
        # # visit link
        # driver.get(h_10_niche)
        # time.sleep(2)
        
        # create global data frame
        overview_data_df = pd.DataFrame()
        top5_monthly_revenue_df = pd.DataFrame()
        
        # # select category
        # time.sleep(2)
        # categories = driver.find_elements_by_class_name("dropdown-toggle")
        # categories[0].click()
        
        
        # # Get catofories names
        # categories_list = driver.find_elements_by_class_name("checkbox")
        # categories_list_names = [x.text for x in categories_list]
        # cleaned_categories = list(filter(None, categories_list_names))
        # # categories_df = pd.DataFrame(cleaned_categories, columns=['categories'])
        
        # type in the key words search
        for words in search:
            #  link for niche
            h_10_niche = "https://members.helium10.com/black-box/niche"
            # visit link
            driver.get(h_10_niche)
            search_bar = driver.find_element_by_id('filter-asin')
            search_bar.clear()
            search_bar.send_keys(words)
            # open categories list
            # categories = driver.find_elements_by_class_name("dropdown-toggle")
            # categories[0].click()
            # time.sleep(1)

    # ----------------remove----------------
            # restrict = 'Select all'

            # for i in range(0,24):
            #     if restrict == categories_list_names[i]:
            #         categories_list = driver.find_elements_by_class_name("checkbox")
            #         categories_list[i].click()
            #     else:
            #         pass
            
            # # close out the toggle
            # categories = driver.find_elements_by_class_name("dropdown-toggle")
            # categories[0].click()
            # click on search button
            search_button = driver.find_elements_by_xpath("//button[contains(text(), 'Search')]")
            search_button[0].click()
            search_button = driver.find_elements_by_xpath("//button[contains(text(), 'Search')]")
            search_button[0].click()

            # Scrape overview data    

            overview_data = driver.find_elements_by_xpath("//div[@class='col-xs-12 col-sm-6 col-md-2']")
            print(len(overview_data))
            
            col1 = []
            col2 = []
            time.sleep(1)
            for i in range(0,5):
                h_10_data = overview_data[i].text
                h_data = h_10_data.split('\n')
                col1.append(h_data[0].strip())
                value = h_data[1].split('$')
                if len(value) > 1:
                    value = value[1]
                    col2.append(value)
                else:
                    col2.append(value[0])
            # pull data from 20-24
            overview_data_df[f'{words}_specs'] = col1
            overview_data_df[f'{words}_value'] = col2

            # Pull revenue data  
            sort_product_list = Select(driver.find_element_by_id('sort'))
            driver.find_element_by_xpath("//select[@name='sort']/option[text()='Monthly Revenue High To Low']").click()

            # pull in data
            # product title
            time.sleep(1)
            product_name = driver.find_elements_by_class_name('media-heading')
            product_names = [name.text for name in product_name]

            # price
            product_price = driver.find_elements_by_class_name('price-chart')
            product_prices = [price.text.split('$')[1] for price in product_price]

            # monthly sales
            product_sales_monthly = driver.find_elements_by_class_name('monthlySales-column')
            product_sales_monthly_s = [sales.text for sales in product_sales_monthly]
            print(len(product_sales_monthly_s))

            # monthly revenue 
            product_revenue_monthly = driver.find_elements_by_class_name('monthlyRevenue-column')
            product_revenue_monthly_s = [rev.text.split('$')[1] for rev in product_revenue_monthly]

            # put into a df
            top5_monthly_revenue_df[f'{words}_product'] = product_names[:5]
            top5_monthly_revenue_df[f'{words}_price'] = product_prices[:5]
            top5_monthly_revenue_df[f'{words}_monthly_sales'] = product_sales_monthly_s[:5]
            top5_monthly_revenue_df[f'{words}_monthly_revenue'] = product_revenue_monthly_s[:5]
        driver.close()
        return [overview_data_df, top5_monthly_revenue_df]
    def etsy(self, search):
        etsy_df = pd.DataFrame()
        for word in search:
            # launch chrome driver
            executable_path = {'executable_path': '/Users/prettyvo/Desktop/chromedriver'}
            browser = Browser('chrome', **executable_path, headless=False)

            base_url = f'https://www.etsy.com'
            browser.visit(base_url)
            html = browser.html
            etsy_html = BeautifulSoup(html, 'html.parser')

            search_url = f'https://www.etsy.com/search?q={word}&explicit=1&order=most_relevant'
            browser.visit(search_url)
            visit_html = browser.html
            word_html = BeautifulSoup(visit_html, 'html.parser')

            # product name
            product = word_html.findAll('div', class_="v2-listing-card__info")
            product_name = [link.h2.text.strip() for link in product]

            # produce price
            price = word_html.findAll('span', class_="n-listing-card__price")
            product_price = [x.text.strip() for x in price]
            # clean data 
            n = [x.split('\n') for x in product_price]
            product_price = [y[0] for y in n]
            b = [x.split('$') for x in product_price]
            locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 
            try:
                product_price = [float(y[1]) for y in b]
            except:
                product_price = [locale.atof(y[1]) for y in b]
            # print(type(product_price[0]))

            # pull in product rating
            rating = word_html.findAll('div', class_="v2-listing-card__shop")
            r_score = [x.span.input for x in rating]
            scores = []
            for x in r_score:
                try:
                    scores.append(float(x['value']))
                except:
                    scores.append(float('0'))
                    
            # df to hold the data    
            etsy_df[f'{word}_name'] = product_name
            etsy_df[f'{word}_price'] = product_price
            etsy_df[f'{word}_rating'] = scores
        browser.quit()
        return etsy_df



## Call class to scrape data

### User input

In [ ]:
searched = []

In [80]:
# example input data from user
keywords = ['candy', 'chocolate','lollipop','bananas','oranges','apples','tacos','kebabs','sandwiches']


In [29]:
for i in range(0, (len(search_words)-1)):
    for j in range(0, (len(searched)-1)):
        if search_words[i] == searched[j]:
            del search_words[i]
        else:
            pass
search_words

['pen']

In [46]:
new_terms = keywords
old_terms = []
# check to see if data is already in database
for i in range(0, (len(keywords)-1)):
    for j in range(0, (len(searched)-1)):
        if keywords[i] == searched[j]:
            old_terms.append(keywords[i])
            del new_terms[i]
        else:
            pass

In [48]:
old_terms

['candy']

In [ ]:
for word in search_words:
    searched.append(word)

In [26]:
searched

['candy',
 'chocolate',
 'lollipop',
 'bananas',
 'oranges',
 'apples',
 'tacos',
 'kebabs',
 'sandwiches']

### Call class and scrape

In [81]:
# call class
class_testing = amazon_h10(keywords)
list_df = class_testing.complete_scrape(keywords)

You searched for ['candy', 'chocolate', 'lollipop', 'bananas', 'oranges', 'apples', 'tacos', 'kebabs', 'sandwiches']
you are scraping amazon
you are scraping helium
you are scraping etsy
scraping complete


In [82]:
# break up df from list returned
search_df = list_df[0]
amazon_df = list_df[1]
revenue_df = list_df[2]
product_data_df = list_df[3]
etsy_df = list_df[4]

In [83]:
etsy_df

,product_name,product_price
search_term,,
candy,Fillable Candy Box - In Case of Emergency Brea...,155.00
candy,Buttered Popcorn Cotton Candy\nChocolateStoryb...,4.95
candy,"Shimmer Rainbow Unicorn Poop Candy! Happy, Edi...",16.00
candy,"Honeycomb candy - Traditional, Cinnamon, or Pe...",18.00 FREE shipping
candy,Candy Cross - Candy Decoration 25/50/100 Piece...,5.00 FREE shipping
chocolate,Vegan Chocolate Assortment Gift Box\nLMChocola...,16.00
chocolate,25 truffle wrappers/chocolate paper holders -F...,11.50 FREE shipping
chocolate,"OMG Toffee, English Toffee, Butter Crunch Dark...",8.00
chocolate,Send a letter written in Chocolate! Ghirardell...,28.95 FREE shipping


In [67]:
# x = revenue_df.set_index(['search_term'])
keywords=['pen']
df = pd.DataFrame()
for x in keywords:
    df = df.append(amazon_df.loc[x])

In [68]:
df

,product_name,product_price
search_term,,
pen,Practical Survival Pen Flashlight Cartridges,17.99
pen,Zebra Pen Retractable Ballpoint 12221,5.63
pen,MyLifeUNIT Fineliner Colored Drawing Assorted,11.37
pen,Fineliner Journaling Calendar Coloring Supplies,24.99
pen,Fisher Space AG7 Original Astronaut,6.29
pen,Ballpoint Cambond Stainless Attendant Retractable,12.19
pen,Wordsworth Black Fountain BLACK GOLD,12.49
pen,Zebra Ballpoint Stainless Retractable Pen,7.67
pen,iMeaniy Ballpoint Signature Colleague Executive,13.49


## Transfer data to database

In [88]:
# Create an engine for the chinook.sqlite database
engine = create_engine("sqlite:///../static/db/top_trends.db", echo=False)
# Declare a Base using `automap_base()`
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
# Base.metadata.create_all(engine)
# create conn
conn = engine.connect()
# To push the objects made and query the server we use a Session object
session = Session(bind=engine)


In [85]:
# add to database
try:
    search_df.to_sql('search', engine, if_exists='append')
    amazon_df.to_sql('amazon', engine, if_exists='append')
    revenue_df.to_sql('total_revenue_h10', engine, if_exists='append')
    product_data_df.to_sql('monthly_revenue_h10', engine, if_exists='append')
    etsy_df.to_sql('etsy', engine, if_exists='append')
except:
    pass


In [89]:
Base.classes.keys()

['amazon',
 'categories',
 'etsy',
 'monthly_revenue_h10',
 'region',
 'search',
 'time',
 'total_revenue_h10']